# Tutorial 05: Eigenvalues and Eigenvectors

Interactive visualizations for understanding eigen-decomposition in ML.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import Axes3D

plt.style.use('seaborn-v0_8-whitegrid')

## 1. What Are Eigenvectors? Geometric Visualization

In [ ]:
def visualize_eigenvectors(A, title="Matrix Transformation"):
    """
    Visualize how a matrix transforms vectors.
    Eigenvectors only get scaled, not rotated!
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Get eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(A)
    
    # Generate unit circle
    theta = np.linspace(0, 2*np.pi, 100)
    circle = np.array([np.cos(theta), np.sin(theta)])
    
    # Transform circle
    ellipse = A @ circle
    
    # Test vectors (some random, some eigenvectors)
    test_vectors = [
        np.array([1, 0]),
        np.array([0, 1]),
        np.array([1, 1]) / np.sqrt(2),
    ]
    
    # Add real eigenvectors
    for i, (val, vec) in enumerate(zip(eigenvalues, eigenvectors.T)):
        if np.isreal(val):
            test_vectors.append(vec.real)
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(test_vectors)))
    
    # Before transformation
    ax1 = axes[0]
    ax1.plot(circle[0], circle[1], 'b-', alpha=0.3, linewidth=2)
    
    for v, c in zip(test_vectors, colors):
        ax1.arrow(0, 0, v[0]*0.9, v[1]*0.9, head_width=0.08, 
                 head_length=0.05, fc=c, ec=c, linewidth=2)
    
    ax1.set_xlim(-2.5, 2.5)
    ax1.set_ylim(-2.5, 2.5)
    ax1.set_aspect('equal')
    ax1.axhline(y=0, color='k', linewidth=0.5)
    ax1.axvline(x=0, color='k', linewidth=0.5)
    ax1.set_title('Before: Unit vectors')
    ax1.grid(True, alpha=0.3)
    
    # After transformation
    ax2 = axes[1]
    ax2.plot(ellipse[0], ellipse[1], 'b-', alpha=0.3, linewidth=2)
    
    for v, c in zip(test_vectors, colors):
        Av = A @ v
        ax2.arrow(0, 0, Av[0]*0.9, Av[1]*0.9, head_width=0.08,
                 head_length=0.05, fc=c, ec=c, linewidth=2)
        
        # Check if it's an eigenvector
        for val, evec in zip(eigenvalues, eigenvectors.T):
            if np.isreal(val):
                evec = evec.real
                if np.allclose(v, evec / np.linalg.norm(evec)) or \
                   np.allclose(v, -evec / np.linalg.norm(evec)):
                    ax2.annotate(f'λ={val.real:.1f}', xy=(Av[0], Av[1]),
                               xytext=(Av[0]+0.3, Av[1]+0.3), fontsize=10)
    
    max_val = max(np.abs(ellipse).max(), 2.5) * 1.1
    ax2.set_xlim(-max_val, max_val)
    ax2.set_ylim(-max_val, max_val)
    ax2.set_aspect('equal')
    ax2.axhline(y=0, color='k', linewidth=0.5)
    ax2.axvline(x=0, color='k', linewidth=0.5)
    ax2.set_title(f'After: Av\nEigenvectors only scale!')
    ax2.grid(True, alpha=0.3)
    
    plt.suptitle(f'{title}\nEigenvalues: {eigenvalues}', fontsize=14)
    plt.tight_layout()
    plt.show()

# Example 1: Scaling matrix
print("Example 1: Diagonal (scaling) matrix")
A = np.array([[2, 0], [0, 0.5]])
visualize_eigenvectors(A, "Scaling Matrix")

# Example 2: Symmetric matrix
print("\nExample 2: Symmetric matrix (orthogonal eigenvectors)")
A = np.array([[3, 1], [1, 2]])
visualize_eigenvectors(A, "Symmetric Matrix")

# Example 3: Shear (only one eigenvector)
print("\nExample 3: Shear matrix (defective - only one eigenvector)")
A = np.array([[1, 1], [0, 1]])
visualize_eigenvectors(A, "Shear Matrix")

## 2. Computing Eigenvalues: Characteristic Polynomial

In [ ]:
def visualize_characteristic_polynomial(A):
    """
    Visualize the characteristic polynomial and its roots (eigenvalues).
    """
    # Get eigenvalues
    eigenvalues = np.linalg.eigvals(A)
    
    # For 2x2: det(A - λI) = λ² - trace(A)λ + det(A)
    trace_A = np.trace(A)
    det_A = np.linalg.det(A)
    
    # Plot characteristic polynomial
    lambda_range = np.linspace(min(eigenvalues.real) - 2, max(eigenvalues.real) + 2, 200)
    
    def char_poly(lam):
        return lam**2 - trace_A * lam + det_A
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.plot(lambda_range, char_poly(lambda_range), 'b-', linewidth=2, 
            label=f'p(λ) = λ² - {trace_A:.1f}λ + {det_A:.1f}')
    ax.axhline(y=0, color='k', linewidth=0.5)
    
    # Mark eigenvalues (roots)
    for val in eigenvalues:
        if np.isreal(val):
            ax.plot(val.real, 0, 'ro', markersize=12)
            ax.annotate(f'λ = {val.real:.2f}', xy=(val.real, 0),
                       xytext=(val.real, 0.5), fontsize=12,
                       arrowprops=dict(arrowstyle='->', color='red'))
    
    ax.set_xlabel('λ', fontsize=14)
    ax.set_ylabel('det(A - λI)', fontsize=14)
    ax.set_title(f'Characteristic Polynomial\nRoots (eigenvalues) where p(λ) = 0')
    ax.legend(fontsize=12)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Matrix A:\n{A}")
    print(f"\nCharacteristic polynomial: λ² - {trace_A:.2f}λ + {det_A:.2f} = 0")
    print(f"Eigenvalues: {eigenvalues}")
    print(f"\nVerification:")
    print(f"  Sum of eigenvalues = {eigenvalues.sum():.2f} = trace(A) = {trace_A:.2f}")
    print(f"  Product of eigenvalues = {eigenvalues.prod():.2f} = det(A) = {det_A:.2f}")

# Example
A = np.array([[4, 2], [1, 3]])
visualize_characteristic_polynomial(A)

## 3. Power Iteration Algorithm

In [ ]:
def power_iteration_visualized(A, max_iter=20):
    """
    Visualize power iteration converging to dominant eigenvector.
    """
    # Start with random vector
    np.random.seed(42)
    x = np.random.randn(2)
    x = x / np.linalg.norm(x)
    
    # Get true eigenvectors for comparison
    eigenvalues, eigenvectors = np.linalg.eig(A)
    dominant_idx = np.argmax(np.abs(eigenvalues))
    dominant_evec = eigenvectors[:, dominant_idx].real
    dominant_eval = eigenvalues[dominant_idx].real
    
    # Store history
    vectors = [x.copy()]
    eigenvalue_estimates = []
    
    for i in range(max_iter):
        y = A @ x
        eval_est = x @ y  # Rayleigh quotient
        eigenvalue_estimates.append(eval_est)
        x = y / np.linalg.norm(y)
        vectors.append(x.copy())
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Left: Vector convergence
    ax1 = axes[0]
    theta = np.linspace(0, 2*np.pi, 100)
    ax1.plot(np.cos(theta), np.sin(theta), 'k--', alpha=0.3)
    
    # Plot iteration vectors with color gradient
    colors = plt.cm.viridis(np.linspace(0, 1, len(vectors)))
    for i, (v, c) in enumerate(zip(vectors, colors)):
        ax1.arrow(0, 0, v[0]*0.9, v[1]*0.9, head_width=0.05, 
                 head_length=0.03, fc=c, ec=c, alpha=0.7)
        if i < 5 or i == len(vectors) - 1:
            ax1.annotate(f'k={i}', xy=(v[0], v[1]), fontsize=8)
    
    # Plot true dominant eigenvector
    ax1.arrow(0, 0, dominant_evec[0]*0.9, dominant_evec[1]*0.9, head_width=0.05,
             head_length=0.03, fc='red', ec='red', linewidth=2, label='True eigenvector')
    
    ax1.set_xlim(-1.5, 1.5)
    ax1.set_ylim(-1.5, 1.5)
    ax1.set_aspect('equal')
    ax1.set_title(f'Vector Convergence\n(color: early → late)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Right: Eigenvalue convergence
    ax2 = axes[1]
    ax2.plot(eigenvalue_estimates, 'b.-', markersize=10, label='Estimate')
    ax2.axhline(y=dominant_eval, color='r', linestyle='--', label=f'True λ = {dominant_eval:.4f}')
    ax2.set_xlabel('Iteration')
    ax2.set_ylabel('Eigenvalue estimate')
    ax2.set_title('Eigenvalue Convergence')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.suptitle(f'Power Iteration on A\nConverges to dominant eigenvalue λ = {dominant_eval:.4f}', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Print convergence rate
    ratio = np.abs(eigenvalues[1-dominant_idx] / eigenvalues[dominant_idx])
    print(f"\nConvergence rate: |λ₂/λ₁| = {ratio:.4f}")
    print(f"Error decreases by factor {ratio:.4f} per iteration")

# Example with well-separated eigenvalues (fast convergence)
print("Example 1: Well-separated eigenvalues (fast convergence)")
A = np.array([[3, 1], [1, 2]])
power_iteration_visualized(A)

# Example with close eigenvalues (slow convergence)
print("\nExample 2: Close eigenvalues (slow convergence)")
A = np.array([[2.1, 0.1], [0.1, 2.0]])
power_iteration_visualized(A, max_iter=50)

## 4. Diagonalization

In [ ]:
def demonstrate_diagonalization(A):
    """
    Show how A = PDP^(-1) works and its use for matrix powers.
    """
    eigenvalues, eigenvectors = np.linalg.eig(A)
    
    P = eigenvectors
    D = np.diag(eigenvalues)
    P_inv = np.linalg.inv(P)
    
    print("Diagonalization: A = P @ D @ P^(-1)")
    print("="*50)
    print(f"\nMatrix A:\n{A}")
    print(f"\nEigenvector matrix P (columns = eigenvectors):\n{P}")
    print(f"\nDiagonal matrix D (eigenvalues on diagonal):\n{D}")
    print(f"\nVerification P @ D @ P^(-1):\n{P @ D @ P_inv}")
    
    # Matrix powers
    print("\n" + "="*50)
    print("Computing A^10 efficiently:")
    print("="*50)
    
    # Method 1: Direct computation
    A_10_direct = np.linalg.matrix_power(A, 10)
    
    # Method 2: Using diagonalization
    D_10 = np.diag(eigenvalues ** 10)
    A_10_diag = P @ D_10 @ P_inv
    
    print(f"\nDirect computation A^10:\n{A_10_direct}")
    print(f"\nUsing diagonalization P @ D^10 @ P^(-1):\n{A_10_diag.real}")
    print(f"\nD^10 (just raise diagonal to 10th power):\n{D_10}")
    
    # Visualize eigenvalue powers
    fig, ax = plt.subplots(figsize=(10, 6))
    
    powers = np.arange(0, 11)
    for i, val in enumerate(eigenvalues):
        if np.isreal(val):
            ax.plot(powers, val.real ** powers, 'o-', 
                   label=f'λ_{i+1}^n = {val.real:.2f}^n', linewidth=2, markersize=8)
    
    ax.set_xlabel('Power n', fontsize=12)
    ax.set_ylabel('λⁿ', fontsize=12)
    ax.set_title('Eigenvalue Powers\nDominant eigenvalue grows fastest!')
    ax.legend(fontsize=12)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Example
A = np.array([[1.5, 0.5], [0.5, 1.5]])
demonstrate_diagonalization(A)

## 5. Symmetric Matrices: Spectral Theorem

In [ ]:
def demonstrate_spectral_theorem(S):
    """
    Show properties of symmetric matrices:
    - Real eigenvalues
    - Orthogonal eigenvectors
    - S = QΛQ^T
    """
    print("Symmetric Matrix Properties")
    print("="*50)
    
    # Verify symmetry
    print(f"\nMatrix S:\n{S}")
    print(f"Is symmetric? {np.allclose(S, S.T)}")
    
    # Eigendecomposition
    eigenvalues, eigenvectors = np.linalg.eigh(S)  # eigh for symmetric
    
    print(f"\nEigenvalues (all real!): {eigenvalues}")
    print(f"\nEigenvectors Q:\n{eigenvectors}")
    
    # Verify orthogonality
    Q = eigenvectors
    print(f"\nQ^T @ Q (should be I):\n{Q.T @ Q}")
    
    # Verify reconstruction
    Lambda = np.diag(eigenvalues)
    S_reconstructed = Q @ Lambda @ Q.T
    print(f"\nReconstruction Q @ Λ @ Q^T:\n{S_reconstructed}")
    
    # Visualize orthogonal eigenvectors
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Draw unit circle
    theta = np.linspace(0, 2*np.pi, 100)
    ax.plot(np.cos(theta), np.sin(theta), 'k--', alpha=0.3)
    
    # Draw eigenvectors
    colors = ['blue', 'red']
    for i, (val, vec) in enumerate(zip(eigenvalues, eigenvectors.T)):
        ax.arrow(0, 0, vec[0]*0.9, vec[1]*0.9, head_width=0.05,
                head_length=0.03, fc=colors[i], ec=colors[i], linewidth=2,
                label=f'v{i+1} (λ={val:.2f})')
    
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')
    ax.set_title('Orthogonal Eigenvectors\n(Perpendicular for symmetric matrices)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.axvline(x=0, color='k', linewidth=0.5)
    
    # Add angle annotation
    angle = np.degrees(np.arccos(np.dot(eigenvectors[:, 0], eigenvectors[:, 1])))
    ax.annotate(f'Angle: {angle:.1f}°', xy=(0.3, 0.3), fontsize=12)
    
    plt.tight_layout()
    plt.show()

# Example
S = np.array([[3, 1], [1, 2]])
demonstrate_spectral_theorem(S)

## 6. ML Application: Principal Component Analysis (PCA)

In [ ]:
def demonstrate_pca():
    """
    Show how PCA uses eigenvalues/eigenvectors of covariance matrix.
    """
    np.random.seed(42)
    
    # Generate correlated 2D data
    n_samples = 200
    mean = [2, 3]
    cov = [[3, 2], [2, 2]]  # Correlated
    data = np.random.multivariate_normal(mean, cov, n_samples)
    
    # Center the data
    data_centered = data - data.mean(axis=0)
    
    # Compute covariance matrix
    cov_matrix = np.cov(data_centered.T)
    
    # Eigendecomposition
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
    
    # Sort by eigenvalue (descending)
    idx = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    print("PCA via Eigendecomposition")
    print("="*50)
    print(f"\nCovariance matrix:\n{cov_matrix}")
    print(f"\nEigenvalues (variances): {eigenvalues}")
    print(f"Explained variance ratio: {eigenvalues / eigenvalues.sum()}")
    print(f"\nPrincipal components (eigenvectors):\n{eigenvectors}")
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # Original data with principal components
    ax1 = axes[0]
    ax1.scatter(data_centered[:, 0], data_centered[:, 1], alpha=0.5)
    
    # Draw principal components (scaled by sqrt of eigenvalue)
    for i, (val, vec) in enumerate(zip(eigenvalues, eigenvectors.T)):
        scale = np.sqrt(val) * 2
        ax1.arrow(0, 0, vec[0]*scale, vec[1]*scale, head_width=0.1,
                 head_length=0.05, fc=f'C{i}', ec=f'C{i}', linewidth=3,
                 label=f'PC{i+1} (var={val:.2f})')
    
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_title('Original Data with PCs\n(PCs are eigenvectors of covariance)')
    ax1.legend()
    ax1.set_aspect('equal')
    ax1.grid(True, alpha=0.3)
    
    # Project onto PC1
    projected = data_centered @ eigenvectors
    
    ax2 = axes[1]
    ax2.scatter(projected[:, 0], projected[:, 1], alpha=0.5)
    ax2.axhline(y=0, color='r', linestyle='--', label='PC2=0 projection')
    ax2.set_xlabel('PC1 score')
    ax2.set_ylabel('PC2 score')
    ax2.set_title('Data in PC coordinates\n(Decorrelated!)')
    ax2.set_aspect('equal')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Scree plot (eigenvalues)
    ax3 = axes[2]
    ax3.bar([1, 2], eigenvalues, color=['C0', 'C1'])
    ax3.set_xlabel('Principal Component')
    ax3.set_ylabel('Eigenvalue (Variance)')
    ax3.set_title('Scree Plot\n(Eigenvalue = variance explained)')
    ax3.set_xticks([1, 2])
    
    for i, val in enumerate(eigenvalues):
        ratio = val / eigenvalues.sum() * 100
        ax3.annotate(f'{ratio:.1f}%', xy=(i+1, val), ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

demonstrate_pca()

## 7. ML Application: PageRank

In [ ]:
def demonstrate_pagerank():
    """
    Show PageRank as an eigenvector problem.
    """
    # Simple web graph
    # Page 0: links to 1, 2
    # Page 1: links to 2
    # Page 2: links to 0
    # Page 3: links to 0, 2
    
    n = 4
    links = [
        [1, 2],      # Page 0 links to
        [2],         # Page 1 links to
        [0],         # Page 2 links to
        [0, 2]       # Page 3 links to
    ]
    
    # Build transition matrix
    M = np.zeros((n, n))
    for i, targets in enumerate(links):
        for j in targets:
            M[j, i] = 1 / len(targets)
    
    print("PageRank as Eigenvector Problem")
    print("="*50)
    print(f"\nTransition matrix M (column stochastic):\n{M}")
    
    # Add damping (teleportation)
    damping = 0.85
    M_pagerank = damping * M + (1 - damping) / n * np.ones((n, n))
    print(f"\nPageRank matrix (with damping {damping}):\n{M_pagerank}")
    
    # Find eigenvector for eigenvalue 1
    eigenvalues, eigenvectors = np.linalg.eig(M_pagerank)
    print(f"\nEigenvalues: {eigenvalues}")
    
    # Eigenvector for eigenvalue 1 is the PageRank
    idx = np.argmax(np.abs(eigenvalues))  # Should be eigenvalue ≈ 1
    pagerank = np.abs(eigenvectors[:, idx])
    pagerank = pagerank / pagerank.sum()  # Normalize
    
    print(f"\nPageRank (eigenvector for λ=1): {pagerank}")
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Graph visualization
    ax1 = axes[0]
    positions = np.array([[0, 1], [1, 1], [1, 0], [0, 0]])
    
    # Draw edges
    for i, targets in enumerate(links):
        for j in targets:
            dx = positions[j, 0] - positions[i, 0]
            dy = positions[j, 1] - positions[i, 1]
            ax1.arrow(positions[i, 0], positions[i, 1], dx*0.8, dy*0.8,
                     head_width=0.05, head_length=0.03, fc='gray', ec='gray', alpha=0.5)
    
    # Draw nodes (size proportional to PageRank)
    sizes = pagerank * 3000
    ax1.scatter(positions[:, 0], positions[:, 1], s=sizes, c='steelblue', zorder=5)
    for i, (pos, rank) in enumerate(zip(positions, pagerank)):
        ax1.annotate(f'{i}\n{rank:.2f}', xy=pos, ha='center', va='center', fontsize=12)
    
    ax1.set_xlim(-0.5, 1.5)
    ax1.set_ylim(-0.5, 1.5)
    ax1.set_aspect('equal')
    ax1.set_title('Web Graph\n(node size = PageRank)')
    ax1.axis('off')
    
    # Power iteration convergence
    ax2 = axes[1]
    ranks = np.ones(n) / n
    history = [ranks.copy()]
    
    for _ in range(20):
        ranks = M_pagerank @ ranks
        history.append(ranks.copy())
    
    history = np.array(history)
    for i in range(n):
        ax2.plot(history[:, i], label=f'Page {i}')
    
    ax2.set_xlabel('Iteration')
    ax2.set_ylabel('Rank')
    ax2.set_title('PageRank Convergence\n(Power iteration on M)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

demonstrate_pagerank()

## 8. Stability Analysis

In [ ]:
def demonstrate_stability():
    """
    Show how eigenvalues determine stability of dynamical systems.
    """
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    systems = [
        ('Stable (|λ| < 1)', np.array([[0.8, 0.1], [0.1, 0.7]])),
        ('Unstable (|λ| > 1)', np.array([[1.1, 0.1], [0.1, 1.05]])),
        ('Oscillating', np.array([[0, -0.9], [0.9, 0]]))  # Complex eigenvalues
    ]
    
    for col, (name, A) in enumerate(systems):
        eigenvalues = np.linalg.eigvals(A)
        spectral_radius = np.max(np.abs(eigenvalues))
        
        # Top row: eigenvalue plot
        ax1 = axes[0, col]
        theta = np.linspace(0, 2*np.pi, 100)
        ax1.plot(np.cos(theta), np.sin(theta), 'k--', alpha=0.5, label='Unit circle')
        ax1.scatter(eigenvalues.real, eigenvalues.imag, s=100, c='red', zorder=5)
        for val in eigenvalues:
            ax1.annotate(f'λ={val:.2f}', xy=(val.real, val.imag), 
                        xytext=(val.real+0.1, val.imag+0.1))
        ax1.set_xlim(-1.5, 1.5)
        ax1.set_ylim(-1.5, 1.5)
        ax1.set_aspect('equal')
        ax1.axhline(y=0, color='k', linewidth=0.5)
        ax1.axvline(x=0, color='k', linewidth=0.5)
        ax1.set_title(f'{name}\nρ(A) = {spectral_radius:.2f}')
        ax1.set_xlabel('Re(λ)')
        ax1.set_ylabel('Im(λ)')
        ax1.legend(loc='upper right')
        ax1.grid(True, alpha=0.3)
        
        # Bottom row: trajectory
        ax2 = axes[1, col]
        
        # Multiple initial conditions
        for _ in range(5):
            x = np.random.randn(2)
            trajectory = [x.copy()]
            for _ in range(30):
                x = A @ x
                trajectory.append(x.copy())
            trajectory = np.array(trajectory)
            ax2.plot(trajectory[:, 0], trajectory[:, 1], '.-', alpha=0.7, markersize=3)
        
        ax2.scatter([0], [0], s=100, c='black', marker='*', zorder=5)
        ax2.set_xlabel('x₁')
        ax2.set_ylabel('x₂')
        ax2.set_title('Trajectories x_{t+1} = Ax_t')
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nStability Rule for discrete systems x_{t+1} = Ax_t:")
    print("- Stable if spectral radius ρ(A) = max|λᵢ| < 1")
    print("- Unstable if any |λᵢ| > 1")
    print("- Complex eigenvalues cause oscillation/rotation")

demonstrate_stability()

## 9. Complex Eigenvalues and Rotation

In [ ]:
def demonstrate_complex_eigenvalues():
    """
    Show how complex eigenvalues relate to rotation.
    """
    # Rotation matrix
    angle = np.pi / 6  # 30 degrees
    R = np.array([[np.cos(angle), -np.sin(angle)],
                  [np.sin(angle), np.cos(angle)]])
    
    eigenvalues = np.linalg.eigvals(R)
    
    print("Rotation Matrix and Complex Eigenvalues")
    print("="*50)
    print(f"\nRotation by {np.degrees(angle):.0f}° matrix:\n{R}")
    print(f"\nEigenvalues: {eigenvalues}")
    print(f"\nEigenvalues in polar form:")
    for val in eigenvalues:
        r = np.abs(val)
        theta = np.angle(val)
        print(f"  {val:.4f} = {r:.4f} * e^(i*{np.degrees(theta):.1f}°)")
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Eigenvalues in complex plane
    ax1 = axes[0]
    theta_circle = np.linspace(0, 2*np.pi, 100)
    ax1.plot(np.cos(theta_circle), np.sin(theta_circle), 'k--', alpha=0.5, label='Unit circle')
    ax1.scatter(eigenvalues.real, eigenvalues.imag, s=100, c='red', zorder=5)
    for val in eigenvalues:
        ax1.annotate(f'λ={val:.2f}', xy=(val.real, val.imag),
                    xytext=(val.real+0.1, val.imag+0.1))
    ax1.set_xlim(-1.5, 1.5)
    ax1.set_ylim(-1.5, 1.5)
    ax1.set_aspect('equal')
    ax1.axhline(y=0, color='k', linewidth=0.5)
    ax1.axvline(x=0, color='k', linewidth=0.5)
    ax1.set_title('Complex Eigenvalues\n(on unit circle for rotation)')
    ax1.set_xlabel('Re(λ)')
    ax1.set_ylabel('Im(λ)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Rotation effect
    ax2 = axes[1]
    
    # Track a vector through multiple rotations
    v = np.array([1, 0])
    vectors = [v.copy()]
    for _ in range(12):
        v = R @ v
        vectors.append(v.copy())
    vectors = np.array(vectors)
    
    ax2.plot(np.cos(theta_circle), np.sin(theta_circle), 'k--', alpha=0.3)
    ax2.plot(vectors[:, 0], vectors[:, 1], 'b.-', markersize=10, linewidth=2)
    ax2.scatter([vectors[0, 0]], [vectors[0, 1]], s=100, c='green', marker='o', zorder=5, label='Start')
    ax2.scatter([vectors[-1, 0]], [vectors[-1, 1]], s=100, c='red', marker='s', zorder=5, label='End')
    
    ax2.set_xlim(-1.5, 1.5)
    ax2.set_ylim(-1.5, 1.5)
    ax2.set_aspect('equal')
    ax2.set_title(f'Repeated Rotation by {np.degrees(angle):.0f}°\n(No real eigenvector stays fixed!)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

demonstrate_complex_eigenvalues()

## 10. Summary

In [ ]:
print("""
KEY CONCEPTS SUMMARY
====================

1. DEFINITION
   - Eigenvector v: direction that only scales under A
   - Eigenvalue λ: the scaling factor
   - Av = λv

2. FINDING EIGENVALUES
   - Solve det(A - λI) = 0 (characteristic polynomial)
   - For large matrices: use power iteration or QR algorithm
   - Sum of eigenvalues = trace(A)
   - Product of eigenvalues = det(A)

3. DIAGONALIZATION
   - A = PDP⁻¹ where P has eigenvectors, D has eigenvalues
   - Makes matrix powers easy: Aⁿ = PDⁿP⁻¹
   - Not all matrices are diagonalizable

4. SYMMETRIC MATRICES (SPECTRAL THEOREM)
   - Always real eigenvalues
   - Orthogonal eigenvectors
   - A = QΛQᵀ (orthogonal diagonalization)

5. ML APPLICATIONS
   - PCA: Principal components are eigenvectors of covariance
   - PageRank: Ranking is eigenvector for λ=1
   - Stability: |λ| < 1 for all eigenvalues ⟹ stable system
   - RNNs: Eigenvalues determine gradient flow

6. COMPLEX EIGENVALUES
   - Indicate rotation in the transformation
   - Come in conjugate pairs for real matrices
   - |λ| determines growth, arg(λ) determines rotation
""")